In [ ]:
#Creation of Model 1

#Features:
#Team Win % Differential: (Team 1 % - Team 2 %) --> Negative for team 1, Positive for Team 2
#Closeness of Team: eclidean distance between both teams --> if lower distance has greater predictive power on team choice,then then customer prefers closer team
#Sentiment of Team: Teams with higher Sentiment Score will have more people choosing them to win




'''Closeness Metric ~ Takes Two Haversine Distances:

#1.) Find Haversine Distance to convert cartestian cordinates (x,y) for polar coordinates (They Coordinates that take into account the spherical shape of the earth)
#2.) Haversine Distance 1 -> How far the user is from Team 1
#3.) Haversine Distance 2 -> How far the user is from Team 2
#4.)Closeness = Haversine Distance 1 - Haversine Distance 2
#5.) This number shows us: If Negative that user is closer to team 2, If Positive user is closer to team 1, If 0 then Teams are same distance away!
'''

model_1_df = classic1_df_train
model_1_df['closeness_ms'] = closeness(userlat=classic1_df_train['CustomerPostalCodeLatitude'],
                                      userlon=classic1_df_train['CustomerPostalCodeLongitude'],
                                      team1lat=classic1_df_train['M_InstitutionLatitude'],
                                      team1lon=classic1_df_train['M_InstitutionLongitude'],
                                      team2lat=classic1_df_train['S_InstitutionLatitude'],
                                      team2lon=classic1_df_train['S_InstitutionLongitude'])


model_1_df['closeness_ew'] = closeness(userlat=classic1_df_train['CustomerPostalCodeLatitude'],
                                      userlon=classic1_df_train['CustomerPostalCodeLongitude'],
                                      team1lat=classic1_df_train['E_InstitutionLatitude'],
                                      team1lon=classic1_df_train['E_InstitutionLongitude'],
                                      team2lat=classic1_df_train['W_InstitutionLatitude'],
                                      team2lon=classic1_df_train['W_InstitutionLongitude'])


'''Finding Team Win % Differential'''
model_1_df['team_win_diff_ms'] = model_1_df['m_win_%'] - model_1_df['s_win_%']
model_1_df['team_win_diff_ew'] = model_1_df['e_win_%'] - model_1_df['w_win_%']


X_model_1_ms = model_1_df.loc[:, ['team_win_diff_ms', 'closeness_ms']]
y_model_1_ms = (model_1_df['SemifinalWinner_South_Midwest'] == model_1_df['RegionWinner_Midwest']).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X_model_1_ms, y_model_1_ms, train_size=.2, random_state= 24)

log_reg_sm = LogisticRegressionCV(cv=5, scoring='recall', n_jobs=-1, refit=False, random_state=24)
log_reg_sm.fit(X_train, y_train)
log_reg_sm.coef_


# Model Recall on Test Data: 0.3278
# Model Accuracy on Test Data: 0.6077


# array([[ 1.89993871e-02, -6.68585823e-05]])


# Model Recall on Test Data: 1.0000
# Model Accuracy on Test Data: 0.632

#Final
# Model Recall on Test Data: 1.0000
# Model Accuracy on Test Data: 0.5695

In [ ]:

y_pred = log_reg_sm.predict(X_test)

# Compute recall
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Recall on Test Data: {recall:.4f}")
print(f'Model Accuracy on Test Data: {accuracy:.4f}')

In [ ]:

X_model_1_ew = model_1_df.loc[:, ['team_win_diff_ew', 'closeness_ew']]
y_model_1_ew = (model_1_df['SemifinalWinner_East_West'] == model_1_df['RegionWinner_East']).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X_model_1_ew, y_model_1_ew, train_size=.2, random_state= 24)

log_reg_ew = LogisticRegressionCV(cv=5, scoring='recall', n_jobs=-1, refit=False, random_state=24)
log_reg_ew.fit(X_train, y_train)
log_reg_ew.coef_

In [ ]:
y_pred = log_reg_ew.predict(X_test)

# Compute recall
recall_ew = recall_score(y_test, y_pred)
accuracy_ew = accuracy_score(y_test, y_pred)

print(f"Model Recall on Test Data: {recall_ew:.4f}")
print(f'Model Accuracy on Test Data: {accuracy_ew:.4f}')

In [ ]:
nat_champ_model_1_df = nat_champ_basic_df = pd.DataFrame({
    "ew_pred_win_%": model_1_df["e_win_%"].where(y_model_1_ew == 1, model_1_df["w_win_%"]), #Return The Win Percentage of the winning team 1 = East, 0 = West
    "ms_pred_win_%": model_1_df["m_win_%"].where(y_model_1_ms == 1, model_1_df["s_win_%"]), #Return The Win Percentage of the winning team 1 = Midwest, 0 = South
    'ew_lat': classic1_df_train['E_InstitutionLatitude'].where(y_model_1_ew == 1, classic1_df_train['W_InstitutionLatitude']),
    'ew_lon':classic1_df_train['E_InstitutionLongitude'].where(y_model_1_ew == 1, classic1_df_train['W_InstitutionLongitude']),
    'ms_lat': classic1_df_train['M_InstitutionLatitude'].where(y_model_1_ms == 1, classic1_df_train['S_InstitutionLatitude']),
    'ms_lon': classic1_df_train['M_InstitutionLongitude'].where(y_model_1_ms == 1, classic1_df_train['S_InstitutionLongitude']),
    "actual_nat_champ": (classic1_df_train['NationalChampion'] == classic1_df_train["SemifinalWinner_East_West"]).astype(int)  # Create a Binary column where 1 = EW Champ, 0 = SM Champ
})



nat_champ_model_1_df['team_win_diff_nat'] =  nat_champ_model_1_df['ew_pred_win_%'] - nat_champ_model_1_df['ms_pred_win_%']


nat_champ_model_1_df['closeness_nat'] = closeness(userlat=classic1_df_train['CustomerPostalCodeLatitude'],
                                userlon=classic1_df_train['CustomerPostalCodeLongitude'],
                                team1lat= nat_champ_model_1_df['ew_lat'],
                                team1lon = nat_champ_model_1_df['ew_lon'],
                                team2lat= nat_champ_model_1_df['ms_lat'],
                                team2lon= nat_champ_model_1_df['ms_lon'])

# nat_champ_model_1_df

X_model_1_nat = nat_champ_model_1_df.loc[:, ['team_win_diff_nat','closeness_nat']]
y_model_1_nat = nat_champ_model_1_df['actual_nat_champ']

X_train, X_test, y_train, y_test = train_test_split(X_model_1_nat, y_model_1_nat, train_size=.2, random_state=24)

log_reg_nat = LogisticRegressionCV(cv=5, scoring='recall', n_jobs=-1, refit=False, random_state=24)
log_reg_nat.fit(X_train, y_train)



In [ ]:
recall_nat = recall_score(y_test, y_pred)
accuracy_nat = accuracy_score(y_test, y_pred)

print(f"Model Recall on Test Data: {recall_nat:.4f}")
print(f'Model Accuracy on Test Data: {accuracy_nat:.4f}')